## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [1]:
from dotenv import load_dotenv
import os
import mysql.connector

load_dotenv()  # Load environment variables from .env file
mysql_password = os.getenv("MYSQL_PASSWORD")

cnx = mysql.connector.connect(
    host = "localhost",
    user = "mirtilo",
    password = mysql_password
)

print(cnx)

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [2]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [3]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [4]:
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)

('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [7]:
import pandas as pd
df_livros = pd.read_csv(r"C:\Users\almeida-mirtilo\Documents\pipeline-python-mongo-mysql\data\tabela_livros.csv")
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,68b6431758d1ebd805bfec68,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,68b6431758d1ebd805bfec69,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,68b6431758d1ebd805bfec7b,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,68b6431758d1ebd805bfec89,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,68b6431758d1ebd805bfec8b,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [8]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [9]:
df_livros.shape

(1484, 13)

In [11]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               
               PRIMARY KEY (id));
""")

**Selecionando a base de dados para verificar a tabela criada**

In [12]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
  print(tb)

('tb_livros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [18]:
for i, row in df_livros.iterrows():
    print(tuple(row))

('68b6431758d1ebd805bfec68', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('68b6431758d1ebd805bfec69', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('68b6431758d1ebd805bfec7b', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('68b6431758d1ebd805bfec89', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('68b6431758d1ebd805bfec8b', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('68b6431758d1ebd805bfec8e', 'Iniciando em programação', 'livros', 61.73, 0.0, '2021-02-01', 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('68b6431758d1ebd805bfec

In [19]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

[('68b6431758d1ebd805bfec68',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('68b6431758d1ebd805bfec69',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('68b6431758d1ebd805bfec7b',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('68b6431758d1ebd805bfec89',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('68b6431758d1ebd805bfec8b',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('68b6431758d1ebd805bfec8e',
  'Iniciando em progr

In [21]:
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados)
cnx.commit()

In [22]:
print(cursor.rowcount, "dados foram inseridos.")

1484 dados foram inseridos.


## Visualizando os dados inseridos

In [25]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros;")

for row in cursor:
    print(row)

('68b6431758d1ebd805bfec68', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('68b6431758d1ebd805bfec69', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('68b6431758d1ebd805bfec7b', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('68b6431758d1ebd805bfec89', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('68b6431758d1ebd805bfec8b', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('68b6431758d1ebd805bfec8e', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

In [26]:
cursor.close()

True

In [27]:
cnx.close()